In [1]:
import pandas as pd
import numpy as np
import os
import json
import itertools as it
import time
import subprocess
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
from datetime import date, datetime, timedelta
from tensorflow import keras
from sklearn.metrics import confusion_matrix,classification_report,ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, average_precision_score
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model
import warnings
warnings.filterwarnings("ignore")

2024-04-24 11:12:54.410516: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-24 11:12:54.652007: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-24 11:12:55.485059: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
majestic = pd.read_csv('./treino/majestic_ascii.csv')
majestic['69'] = 0
majestic['70'] = 0
majestic['71'] = 0
majestic

,domain,label,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,70,71
0,google.com,0,103,111,111,103,108,101,46,99,...,0,0,0,0,0,0,0,0,0,0
1,facebook.com,0,102,97,99,101,98,111,111,107,...,0,0,0,0,0,0,0,0,0,0
2,youtube.com,0,121,111,117,116,117,98,101,46,...,0,0,0,0,0,0,0,0,0,0
3,twitter.com,0,116,119,105,116,116,101,114,46,...,0,0,0,0,0,0,0,0,0,0
4,instagram.com,0,105,110,115,116,97,103,114,97,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,fidgit.com,0,102,105,100,103,105,116,46,99,...,0,0,0,0,0,0,0,0,0,0
999996,fixwindows10connections.com,0,102,105,120,119,105,110,100,111,...,0,0,0,0,0,0,0,0,0,0
999997,flyerguide.com,0,102,108,121,101,114,103,117,105,...,0,0,0,0,0,0,0,0,0,0
999998,forwardpathway.com,0,102,111,114,119,97,114,100,112,...,0,0,0,0,0,0,0,0,0,0


In [3]:
dga = pd.read_csv('./treino/df_train.csv')
dga = dga.drop('family', axis = 1)
dga

,domain,label,3,4,5,6,7,8,9,10,...,63,64,65,66,67,68,69,70,71,72
0,uzwdmrlhr.tk,1,117,122,119,100,109,114,108,104,...,0,0,0,0,0,0,0,0,0,0
1,uzwdmrlhr.com,1,117,122,119,100,109,114,108,104,...,0,0,0,0,0,0,0,0,0,0
2,uzwdmrlhr.pages.dev,1,117,122,119,100,109,114,108,104,...,0,0,0,0,0,0,0,0,0,0
3,rhrlwdumz.tk,1,114,104,114,108,119,100,117,109,...,0,0,0,0,0,0,0,0,0,0
4,rhrlwdumz.com,1,114,104,114,108,119,100,117,109,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1352254,kwgjjpbcaaanbivklyxx.com,1,107,119,103,106,106,112,98,99,...,0,0,0,0,0,0,0,0,0,0
1352255,vxijfvgvihjnfvofxmyc.com,1,118,120,105,106,102,118,103,118,...,0,0,0,0,0,0,0,0,0,0
1352256,cepswmbgtknhtkhlsecg.com,1,99,101,112,115,119,109,98,103,...,0,0,0,0,0,0,0,0,0,0
1352257,tifjhjtouqmdlbkfnuow.com,1,116,105,102,106,104,106,116,111,...,0,0,0,0,0,0,0,0,0,0


In [4]:
dga.columns = majestic.columns

In [5]:
dga = dga.loc[~dga['domain'].isin(majestic['domain'])].reset_index(drop=True)

In [6]:
df_treino = pd.concat([majestic.sample(500000, random_state=1),dga], axis = 0, ignore_index=True)
df_treino

,domain,label,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,70,71
0,wwoof.fr,0,119,119,111,111,102,46,102,114,...,0,0,0,0,0,0,0,0,0,0
1,digimanthan.com,0,100,105,103,105,109,97,110,116,...,0,0,0,0,0,0,0,0,0,0
2,injo.com,0,105,110,106,111,46,99,111,109,...,0,0,0,0,0,0,0,0,0,0
3,aclanthology.info,0,97,99,108,97,110,116,104,111,...,0,0,0,0,0,0,0,0,0,0
4,aseabreeze.co.uk,0,97,115,101,97,98,114,101,101,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1852202,kwgjjpbcaaanbivklyxx.com,1,107,119,103,106,106,112,98,99,...,0,0,0,0,0,0,0,0,0,0
1852203,vxijfvgvihjnfvofxmyc.com,1,118,120,105,106,102,118,103,118,...,0,0,0,0,0,0,0,0,0,0
1852204,cepswmbgtknhtkhlsecg.com,1,99,101,112,115,119,109,98,103,...,0,0,0,0,0,0,0,0,0,0
1852205,tifjhjtouqmdlbkfnuow.com,1,116,105,102,106,104,106,116,111,...,0,0,0,0,0,0,0,0,0,0


In [7]:
X_train = df_treino.drop(['domain','label'], axis = 1)
y_train = df_treino['label']

In [8]:
X_train = X_train.to_numpy()
y_train = y_train.to_numpy()

In [9]:
X_train.shape

(1852207, 70)

In [10]:
#Carregando hyperparâmetros de regularizadores, otimizador e função de ativação
l1l2 = keras.regularizers.l1_l2(l1=0.00001, l2=0.0001)
opt = keras.optimizers.Adam(learning_rate=0.0001)
fx = "relu"

2024-04-24 11:13:30.146595: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-24 11:13:30.331742: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-24 11:13:30.331899: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [11]:
#Estrutura do modelo convolucional profundo
model_bin = tf.keras.Sequential([
      
        layers.Embedding(input_dim=256,output_dim=60,input_length=70),
    
        layers.Conv1D(500, 7, activation=fx, activity_regularizer=l1l2, padding='same'),
        layers.ReLU(),
            
        layers.Conv1D(250, 6, activation=fx, activity_regularizer=l1l2, padding='same'),
        layers.ReLU(),
        layers.MaxPooling1D(pool_size=6, strides=1, padding='same'),
    
        layers.Conv1D(120, 5, activation=fx, activity_regularizer=l1l2, padding='same'),
        layers.ReLU(),
    
        layers.Conv1D(60, 5, activation=fx, activity_regularizer=l1l2, padding='same'),
        layers.ReLU(),
        layers.MaxPooling1D(pool_size=4, strides=1, padding='same'),
    
        layers.Conv1D(30, 4, activation=fx, activity_regularizer=l1l2, padding='same'),
        layers.ReLU(),
    
        layers.Conv1D(30, 3, activation=fx, activity_regularizer=l1l2, padding='same'),
        layers.ReLU(),
        layers.MaxPooling1D(pool_size=3, strides=1, padding='same'),
    
        layers.Flatten(),
        layers.Dense(15, activation=fx, activity_regularizer=l1l2),
        layers.Dense(1, activation='sigmoid')])

model_bin.compile(optimizer = opt, loss = 'binary_crossentropy', metrics='accuracy')
model_bin.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 70, 60)            15360     
                                                                 
 conv1d (Conv1D)             (None, 70, 500)           210500    
                                                                 
 re_lu (ReLU)                (None, 70, 500)           0         
                                                                 
 conv1d_1 (Conv1D)           (None, 70, 250)           750250    
                                                                 
 re_lu_1 (ReLU)              (None, 70, 250)           0         
                                                                 
 max_pooling1d (MaxPooling1  (None, 70, 250)           0         
 D)                                                              
                                                        

In [13]:
best = tf.keras.callbacks.ModelCheckpoint(
    filepath='best_model_acc',
    save_weights_only=False,
    monitor='accuracy',
    mode='max',
    save_best_only=True)

In [12]:
# Ajustando uso de memoria RAM e GPU
physical_devices = tf.config.list_physical_devices('GPU')
try:
  tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
  pass

In [ ]:
model_bin.fit(X_train, y_train,
              epochs=10,
              callbacks = [best],
              validation_split = 0.2,
              batch_size=200)

Epoch 1/10


2024-01-22 16:02:07.688802: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8902
2024-01-22 16:02:08.330287: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-01-22 16:02:08.340279: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f2f24004a10 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-01-22 16:02:08.340297: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Laptop GPU, Compute Capability 8.6
2024-01-22 16:02:08.344146: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-01-22 16:02:08.423708: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifeti

7408/7409 [============================>.] - ETA: 0s - loss: 0.1030 - accuracy: 0.9635

2024-01-22 16:06:51.466106: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.24GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-01-22 16:06:51.616001: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.86GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-01-22 16:06:51.736161: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.70GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-01-22 16:06:51.882202: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU

7409/7409 [==============================] - ETA: 0s - loss: 0.1030 - accuracy: 0.9635

...

INFO:tensorflow:Assets written to: best_model_acc/assets


INFO:tensorflow:Assets written to: best_model_acc/assets


7409/7409 [==============================] - 309s 41ms/step - loss: 0.1030 - accuracy: 0.9635 - val_loss: 0.0805 - val_accuracy: 0.9732
Epoch 2/10
7409/7409 [==============================] - ETA: 0s - loss: 0.0577 - accuracy: 0.9806INFO:tensorflow:Assets written to: best_model_acc/assets


INFO:tensorflow:Assets written to: best_model_acc/assets


7409/7409 [==============================] - 300s 40ms/step - loss: 0.0577 - accuracy: 0.9806 - val_loss: 0.0874 - val_accuracy: 0.9693
Epoch 3/10
5197/7409 [====================>.........] - ETA: 1:23 - loss: 0.0461 - accuracy: 0.9847

In [16]:
model_bin.load_weights('best_model_acc')

2024-01-18 15:54:44.070243: W tensorflow/core/util/tensor_slice_reader.cc:97] Could not open best_model: FAILED_PRECONDITION: best_model; É um diretório: perhaps your file is in a different file format and you need to use a different restore operator?


In [12]:
model_bin = tf.keras.models.load_model('best_model')

In [16]:
legit = majestic.loc[~majestic['domain'].isin(df_treino['domain'])].reset_index(drop=True)

## Testes Treinamento Inicial

In [17]:
model_bin = tf.keras.models.load_model('best_model')

for i in range (0,12):
    dga = pd.read_csv('./teste/df_teste'+str(i)+'.csv')
    dga = dga.drop('family', axis = 1)
    dga = dga.loc[~dga['domain'].isin(majestic['domain'])].reset_index(drop=True)

    dga.columns = legit.columns
    df_teste = pd.concat([legit,dga], axis = 0, ignore_index=True)

    X_test = df_teste.drop(['domain','label'], axis = 1)
    y_test = df_teste['label']

    X_test = X_test.to_numpy()
    y_test = y_test.to_numpy()

    with tf.device('/cpu:0'):
        predictions = model_bin.predict(X_test)
        predictions = np.where(predictions > 0.5, 1, 0)
    
    print('Matriz de Confusão Teste: ', str(i))
    print(confusion_matrix(y_test,predictions))

    print('Relatório de Classificação')
    print(classification_report(y_test,predictions, digits=4))
    print('____________________')

60669/60669 [==============================] - 713s 12ms/step
Matriz de Confusão Teste:  0
[[ 483293   16707]
 [  20437 1420944]]
Relatório de Classificação
              precision    recall  f1-score   support

           0     0.9594    0.9666    0.9630    500000
           1     0.9884    0.9858    0.9871   1441381

    accuracy                         0.9809   1941381
   macro avg     0.9739    0.9762    0.9750   1941381
weighted avg     0.9809    0.9809    0.9809   1941381

____________________
57413/57413 [==============================] - 684s 12ms/step
Matriz de Confusão Teste:  1
[[ 483293   16707]
 [  20592 1316606]]
Relatório de Classificação
              precision    recall  f1-score   support

           0     0.9591    0.9666    0.9628    500000
           1     0.9875    0.9846    0.9860   1337198

    accuracy                         0.9797   1837198
   macro avg     0.9733    0.9756    0.9744   1837198
weighted avg     0.9798    0.9797    0.9797   1837198

___________

## Teste Primeiro Treinamento Incremental

In [13]:
## Predicting train dataset
#with tf.device('/cpu:0'):
predictions = model_bin.predict(X_train)
predictions = np.where(predictions > 0.5, 1, 0)
    
print('Matriz de Confusão Teste:')
print(confusion_matrix(y_train,predictions))

2024-04-24 11:14:11.559707: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8902


   69/57882 [..............................] - ETA: 2:09

2024-04-24 11:14:12.173512: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


57882/57882 [==============================] - 123s 2ms/step
Matriz de Confusão Teste:
[[ 490840    9160]
 [  11531 1340676]]


In [14]:
df_treino['predict'] = predictions

In [15]:
acertos = df_treino[df_treino['label']==df_treino['predict']]

In [16]:
acertos = acertos[acertos['label']==1]

In [17]:
acertos = acertos.sample(frac = 0.01, random_state = 1)

In [18]:
erros = df_treino[df_treino['label']!=df_treino['predict']]

In [19]:
erros = erros[erros['label']==1]

In [20]:
erros = erros.drop('predict', axis=1)

In [21]:
acertos = acertos.drop('predict', axis=1)

In [22]:
acertos

,domain,label,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,70,71
1370229,amuxea.com,1,97,109,117,120,101,97,46,99,...,0,0,0,0,0,0,0,0,0,0
1120403,j83cc9021d7316c133cf37eb93d09c03e7.in,1,106,56,51,99,99,57,48,50,...,0,0,0,0,0,0,0,0,0,0
945004,okhimmqqldtw.biz,1,111,107,104,105,109,109,113,113,...,0,0,0,0,0,0,0,0,0,0
910894,sirudis.info,1,115,105,114,117,100,105,115,46,...,0,0,0,0,0,0,0,0,0,0
1676666,ntzjiexqpx.org,1,110,116,122,106,105,101,120,113,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1477597,ytfqzv.com,1,121,116,102,113,122,118,46,99,...,0,0,0,0,0,0,0,0,0,0
703762,ymyumwqsmqmqmaui.xyz,1,121,109,121,117,109,119,113,115,...,0,0,0,0,0,0,0,0,0,0
726220,gugguwagqocycegk.xyz,1,103,117,103,103,117,119,97,103,...,0,0,0,0,0,0,0,0,0,0
629531,qdctlgqakpppbwv.su,1,113,100,99,116,108,103,113,97,...,0,0,0,0,0,0,0,0,0,0


In [56]:
majestic = pd.read_csv('./treino/majestic_ascii.csv')
majestic['69'] = 0
majestic['70'] = 0
majestic['71'] = 0

In [57]:
dga_novo = pd.read_csv('./teste/df_teste0.csv')
dga_novo = dga_novo.drop('family', axis = 1)

In [58]:
dga_novo.columns = majestic.columns
dga_novo = dga_novo.loc[~dga_novo['domain'].isin(df_treino['domain'])].reset_index(drop=True)
dga_novo

,domain,label,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,70,71
0,pgbawmrow.tk,1,112,103,98,97,119,109,114,111,...,0,0,0,0,0,0,0,0,0,0
1,pgbawmrow.com,1,112,103,98,97,119,109,114,111,...,0,0,0,0,0,0,0,0,0,0
2,pgbawmrow.pages.dev,1,112,103,98,97,119,109,114,111,...,0,0,0,0,0,0,0,0,0,0
3,womrbapwg.tk,1,119,111,109,114,98,97,112,119,...,0,0,0,0,0,0,0,0,0,0
4,womrbapwg.com,1,119,111,109,114,98,97,112,119,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
868639,fpbsexqdvqkjjrsbcgru.com,1,102,112,98,115,101,120,113,100,...,0,0,0,0,0,0,0,0,0,0
868640,uoymswsdamhrluobcgac.com,1,117,111,121,109,115,119,115,100,...,0,0,0,0,0,0,0,0,0,0
868641,ilnhukbctytccxlsgdbf.com,1,105,108,110,104,117,107,98,99,...,0,0,0,0,0,0,0,0,0,0
868642,fwxvahsxtqbxklxgtqyg.com,1,102,119,120,118,97,104,115,120,...,0,0,0,0,0,0,0,0,0,0


In [23]:
model_bin = tf.keras.models.load_model('best_model')

majestic = pd.read_csv('./treino/majestic_ascii.csv')
majestic['69'] = 0
majestic['70'] = 0
majestic['71'] = 0

dga_novo = pd.read_csv('./teste/df_teste0.csv')
dga_novo = dga_novo.drop('family', axis = 1)

dga_novo.columns = majestic.columns
dga_novo = dga_novo.loc[~dga_novo['domain'].isin(df_treino['domain'])].reset_index(drop=True)

df_treino = pd.concat([majestic.sample(500000, random_state=1),dga_novo,erros,acertos], axis = 0, ignore_index=True)

X_train = df_treino.drop(['domain','label'], axis = 1)
y_train = df_treino['label']

X_train = X_train.to_numpy()
y_train = y_train.to_numpy()

best = tf.keras.callbacks.ModelCheckpoint(
    filepath='best_model_fase00',
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

#Carregando hyperparâmetros de regularizadores, otimizador e função de ativação
l1l2 = keras.regularizers.l1_l2(l1=0.000001, l2=0.00001)
opt = keras.optimizers.Adam(learning_rate=0.00001)
fx = "relu"

model_bin.compile(optimizer = opt, loss = 'binary_crossentropy', metrics='accuracy')

model_bin.fit(X_train, y_train,
              epochs=10,
              callbacks = [best],
              validation_split = 0.2,
              batch_size=200)

Epoch 1/10


2024-04-24 11:17:23.592273: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5c996bb3dc80 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-04-24 11:17:23.592292: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Laptop GPU, Compute Capability 8.6
2024-04-24 11:17:23.627970: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-04-24 11:17:23.836991: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2024-04-24 11:17:24.254228: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.50GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more me

5575/5575 [==============================] - ETA: 0s - loss: 0.0484 - accuracy: 0.9840

2024-04-24 11:20:54.451543: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 7.76GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-04-24 11:20:55.338279: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.73GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-04-24 11:20:55.343885: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.73GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-04-24 11:20:55.386715: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU

INFO:tensorflow:Assets written to: best_model_fase00/assets


...

5575/5575 [==============================] - 224s 40ms/step - loss: 0.0347 - accuracy: 0.9894 - val_loss: 0.1225 - val_accuracy: 0.9548
Epoch 4/10
5575/5575 [==============================] - ETA: 0s - loss: 0.0306 - accuracy: 0.9911INFO:tensorflow:Assets written to: best_model_fase00/assets


INFO:tensorflow:Assets written to: best_model_fase00/assets


5575/5575 [==============================] - 224s 40ms/step - loss: 0.0306 - accuracy: 0.9911 - val_loss: 0.1131 - val_accuracy: 0.9592
Epoch 5/10
5575/5575 [==============================] - 222s 40ms/step - loss: 0.0269 - accuracy: 0.9926 - val_loss: 0.1280 - val_accuracy: 0.9547
Epoch 6/10
5575/5575 [==============================] - ETA: 0s - loss: 0.0235 - accuracy: 0.9941INFO:tensorflow:Assets written to: best_model_fase00/assets


INFO:tensorflow:Assets written to: best_model_fase00/assets


5575/5575 [==============================] - 224s 40ms/step - loss: 0.0235 - accuracy: 0.9941 - val_loss: 0.1162 - val_accuracy: 0.9615
Epoch 7/10
5575/5575 [==============================] - 223s 40ms/step - loss: 0.0202 - accuracy: 0.9954 - val_loss: 0.1340 - val_accuracy: 0.9580
Epoch 8/10
5575/5575 [==============================] - 223s 40ms/step - loss: 0.0172 - accuracy: 0.9965 - val_loss: 0.1435 - val_accuracy: 0.9570
Epoch 9/10
5575/5575 [==============================] - 223s 40ms/step - loss: 0.0148 - accuracy: 0.9974 - val_loss: 0.1526 - val_accuracy: 0.9552
Epoch 10/10
5575/5575 [==============================] - 223s 40ms/step - loss: 0.0127 - accuracy: 0.9980 - val_loss: 0.1640 - val_accuracy: 0.9554


In [ ]:
model_bin = tf.keras.models.load_model('best_model_fase00')
legit = majestic.loc[~majestic['domain'].isin(df_treino['domain'])].reset_index(drop=True)

for i in range (1,12):
    dga = pd.read_csv('./teste/df_teste'+str(i)+'.csv')
    dga = dga.drop('family', axis = 1)
    dga = dga.loc[~dga['domain'].isin(majestic['domain'])].reset_index(drop=True)

    dga.columns = legit.columns
    df_teste = pd.concat([legit,dga], axis = 0, ignore_index=True)

    X_test = df_teste.drop(['domain','label'], axis = 1)
    y_test = df_teste['label']

    X_test = X_test.to_numpy()
    y_test = y_test.to_numpy()

    with tf.device('/cpu:0'):
        predictions = model_bin.predict(X_test)
        predictions = np.where(predictions > 0.5, 1, 0)
    
    print('Matriz de Confusão Teste: ', str(i))
    print(confusion_matrix(y_test,predictions))

    print('Relatório de Classificação')
    print(classification_report(y_test,predictions, digits=4))
    print('____________________')

57412/57412 [==============================] - 675s 12ms/step
Matriz de Confusão Teste:  1
[[ 482614   17364]
 [  16143 1321055]]
Relatório de Classificação
              precision    recall  f1-score   support

           0     0.9676    0.9653    0.9665    499978
           1     0.9870    0.9879    0.9875   1337198

    accuracy                         0.9818   1837176
   macro avg     0.9773    0.9766    0.9770   1837176
weighted avg     0.9817    0.9818    0.9818   1837176

____________________
60791/60791 [==============================] - 693s 11ms/step
Matriz de Confusão Teste:  2
[[ 482614   17364]
 [  19530 1425784]]
Relatório de Classificação
              precision    recall  f1-score   support

           0     0.9611    0.9653    0.9632    499978
           1     0.9880    0.9865    0.9872   1445314

    accuracy                         0.9810   1945292
   macro avg     0.9745    0.9759    0.9752   1945292
weighted avg     0.9811    0.9810    0.9810   1945292

___________

## Teste Segundo Treinamento Incremental

In [7]:
model_bin = tf.keras.models.load_model('best_model_fase0')

majestic = pd.read_csv('./treino/majestic_ascii.csv')
majestic['69'] = 0
majestic['70'] = 0
majestic['71'] = 0

dga = pd.read_csv('./teste/df_teste1.csv')
dga = dga.drop('family', axis = 1)

dga.columns = majestic.columns
dga = dga.loc[~dga['domain'].isin(majestic['domain'])].reset_index(drop=True)

df_treino = pd.concat([majestic.sample(500000, random_state=1),dga], axis = 0, ignore_index=True)

X_train = df_treino.drop(['domain','label'], axis = 1)
y_train = df_treino['label']

X_train = X_train.to_numpy()
y_train = y_train.to_numpy()

best = tf.keras.callbacks.ModelCheckpoint(
    filepath='best_model_fase1',
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

model_bin.fit(X_train, y_train,
              epochs=10,
              callbacks = [best],
              validation_split = 0.2,
              batch_size=200)

2024-01-20 17:39:03.682765: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-20 17:39:03.868840: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-20 17:39:03.868991: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Epoch 1/10


2024-01-20 17:39:13.820285: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8902
2024-01-20 17:39:14.635272: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-01-20 17:39:14.659227: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x559f251e2540 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-01-20 17:39:14.659244: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Laptop GPU, Compute Capability 8.6
2024-01-20 17:39:14.695401: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-01-20 17:39:14.876710: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifeti

7348/7349 [============================>.] - ETA: 0s - loss: 0.0352 - accuracy: 0.9882

2024-01-20 17:43:54.682646: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.19GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-01-20 17:43:54.837139: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.78GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-01-20 17:43:54.963146: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.54GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-01-20 17:43:55.108188: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU

...

INFO:tensorflow:Assets written to: best_model_fase1/assets


INFO:tensorflow:Assets written to: best_model_fase1/assets


7349/7349 [==============================] - 307s 41ms/step - loss: 0.0352 - accuracy: 0.9882 - val_loss: 0.0646 - val_accuracy: 0.9762
Epoch 2/10
7349/7349 [==============================] - ETA: 0s - loss: 0.0282 - accuracy: 0.9907INFO:tensorflow:Assets written to: best_model_fase1/assets


INFO:tensorflow:Assets written to: best_model_fase1/assets


7349/7349 [==============================] - 299s 41ms/step - loss: 0.0282 - accuracy: 0.9907 - val_loss: 0.0494 - val_accuracy: 0.9816
Epoch 3/10
7349/7349 [==============================] - 300s 41ms/step - loss: 0.0234 - accuracy: 0.9925 - val_loss: 0.0555 - val_accuracy: 0.9792
Epoch 4/10
7349/7349 [==============================] - 302s 41ms/step - loss: 0.0197 - accuracy: 0.9938 - val_loss: 0.0737 - val_accuracy: 0.9739
Epoch 5/10
7349/7349 [==============================] - 304s 41ms/step - loss: 0.0167 - accuracy: 0.9950 - val_loss: 0.0903 - val_accuracy: 0.9717
Epoch 6/10
7349/7349 [==============================] - 302s 41ms/step - loss: 0.0146 - accuracy: 0.9956 - val_loss: 0.0860 - val_accuracy: 0.9740
Epoch 7/10
7349/7349 [==============================] - 303s 41ms/step - loss: 0.0128 - accuracy: 0.9962 - val_loss: 0.0834 - val_accuracy: 0.9751
Epoch 8/10
7349/7349 [==============================] - 300s 41ms/step - loss: 0.0115 - accuracy: 0.9967 - val_loss: 0.0847 - val

In [8]:
model_bin = tf.keras.models.load_model('best_model_fase1')
legit = majestic.loc[~majestic['domain'].isin(df_treino['domain'])].reset_index(drop=True)

for i in range (2,12):
    dga = pd.read_csv('./teste/df_teste'+str(i)+'.csv')
    dga = dga.drop('family', axis = 1)
    dga = dga.loc[~dga['domain'].isin(majestic['domain'])].reset_index(drop=True)

    dga.columns = legit.columns
    df_teste = pd.concat([legit,dga], axis = 0, ignore_index=True)

    X_test = df_teste.drop(['domain','label'], axis = 1)
    y_test = df_teste['label']

    X_test = X_test.to_numpy()
    y_test = y_test.to_numpy()

    with tf.device('/cpu:0'):
        predictions = model_bin.predict(X_test)
        predictions = np.where(predictions > 0.5, 1, 0)
    
    print('Matriz de Confusão Teste: ', str(i))
    print(confusion_matrix(y_test,predictions))

    print('Relatório de Classificação')
    print(classification_report(y_test,predictions, digits=4))
    print('____________________')

60792/60792 [==============================] - 774s 13ms/step
Matriz de Confusão Teste:  2
[[ 484046   15954]
 [  16789 1428525]]
Relatório de Classificação
              precision    recall  f1-score   support

           0     0.9665    0.9681    0.9673    500000
           1     0.9890    0.9884    0.9887   1445314

    accuracy                         0.9832   1945314
   macro avg     0.9777    0.9782    0.9780   1945314
weighted avg     0.9832    0.9832    0.9832   1945314

____________________
56590/56590 [==============================] - 695s 12ms/step
Matriz de Confusão Teste:  3
[[ 484046   15954]
 [  20609 1290258]]
Relatório de Classificação
              precision    recall  f1-score   support

           0     0.9592    0.9681    0.9636    500000
           1     0.9878    0.9843    0.9860   1310867

    accuracy                         0.9798   1810867
   macro avg     0.9735    0.9762    0.9748   1810867
weighted avg     0.9799    0.9798    0.9798   1810867

___________

## Teste Terceiro Treino Incremental

In [9]:
model_bin = tf.keras.models.load_model('best_model_fase1')

majestic = pd.read_csv('./treino/majestic_ascii.csv')
majestic['69'] = 0
majestic['70'] = 0
majestic['71'] = 0

dga = pd.read_csv('./teste/df_teste2.csv')
dga = dga.drop('family', axis = 1)

dga.columns = majestic.columns
dga = dga.loc[~dga['domain'].isin(majestic['domain'])].reset_index(drop=True)

df_treino = pd.concat([majestic.sample(500000, random_state=1),dga], axis = 0, ignore_index=True)

X_train = df_treino.drop(['domain','label'], axis = 1)
y_train = df_treino['label']

X_train = X_train.to_numpy()
y_train = y_train.to_numpy()

best = tf.keras.callbacks.ModelCheckpoint(
    filepath='best_model_fase2',
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

model_bin.fit(X_train, y_train,
              epochs=10,
              callbacks = [best],
              validation_split = 0.2,
              batch_size=200)

Epoch 1/10
7782/7782 [==============================] - ETA: 0s - loss: 0.0290 - accuracy: 0.9905INFO:tensorflow:Assets written to: best_model_fase2/assets


INFO:tensorflow:Assets written to: best_model_fase2/assets


7782/7782 [==============================] - 317s 40ms/step - loss: 0.0290 - accuracy: 0.9905 - val_loss: 0.0609 - val_accuracy: 0.9760
Epoch 2/10
7781/7782 [============================>.] - ETA: 0s - loss: 0.0229 - accuracy: 0.9927INFO:tensorflow:Assets written to: best_model_fase2/assets


INFO:tensorflow:Assets written to: best_model_fase2/assets


7782/7782 [==============================] - 314s 40ms/step - loss: 0.0229 - accuracy: 0.9927 - val_loss: 0.0545 - val_accuracy: 0.9786
Epoch 3/10
7782/7782 [==============================] - 314s 40ms/step - loss: 0.0189 - accuracy: 0.9942 - val_loss: 0.0658 - val_accuracy: 0.9757
Epoch 4/10
7782/7782 [==============================] - 314s 40ms/step - loss: 0.0160 - accuracy: 0.9951 - val_loss: 0.0679 - val_accuracy: 0.9770
Epoch 5/10
7781/7782 [============================>.] - ETA: 0s - loss: 0.0136 - accuracy: 0.9960INFO:tensorflow:Assets written to: best_model_fase2/assets


INFO:tensorflow:Assets written to: best_model_fase2/assets


7782/7782 [==============================] - 316s 41ms/step - loss: 0.0136 - accuracy: 0.9960 - val_loss: 0.0608 - val_accuracy: 0.9803
Epoch 6/10
7782/7782 [==============================] - 315s 40ms/step - loss: 0.0119 - accuracy: 0.9965 - val_loss: 0.0644 - val_accuracy: 0.9803
Epoch 7/10
7782/7782 [==============================] - 315s 40ms/step - loss: 0.0106 - accuracy: 0.9969 - val_loss: 0.1042 - val_accuracy: 0.9714
Epoch 8/10
7782/7782 [==============================] - 315s 40ms/step - loss: 0.0096 - accuracy: 0.9973 - val_loss: 0.0821 - val_accuracy: 0.9783
Epoch 9/10
7781/7782 [============================>.] - ETA: 0s - loss: 0.0088 - accuracy: 0.9976INFO:tensorflow:Assets written to: best_model_fase2/assets


INFO:tensorflow:Assets written to: best_model_fase2/assets


7782/7782 [==============================] - 316s 41ms/step - loss: 0.0088 - accuracy: 0.9976 - val_loss: 0.0670 - val_accuracy: 0.9827
Epoch 10/10
7782/7782 [==============================] - 315s 40ms/step - loss: 0.0081 - accuracy: 0.9978 - val_loss: 0.0727 - val_accuracy: 0.9807


In [10]:
model_bin = tf.keras.models.load_model('best_model_fase2')
legit = majestic.loc[~majestic['domain'].isin(df_treino['domain'])].reset_index(drop=True)

for i in range (3,12):
    dga = pd.read_csv('./teste/df_teste'+str(i)+'.csv')
    dga = dga.drop('family', axis = 1)
    dga = dga.loc[~dga['domain'].isin(majestic['domain'])].reset_index(drop=True)

    dga.columns = legit.columns
    df_teste = pd.concat([legit,dga], axis = 0, ignore_index=True)

    X_test = df_teste.drop(['domain','label'], axis = 1)
    y_test = df_teste['label']

    X_test = X_test.to_numpy()
    y_test = y_test.to_numpy()

    with tf.device('/cpu:0'):
        predictions = model_bin.predict(X_test)
        predictions = np.where(predictions > 0.5, 1, 0)
    
    print('Matriz de Confusão Teste: ', str(i))
    print(confusion_matrix(y_test,predictions))

    print('Relatório de Classificação')
    print(classification_report(y_test,predictions, digits=4))
    print('____________________')

56590/56590 [==============================] - 707s 12ms/step
Matriz de Confusão Teste:  3
[[ 481841   18159]
 [  17110 1293757]]
Relatório de Classificação
              precision    recall  f1-score   support

           0     0.9657    0.9637    0.9647    500000
           1     0.9862    0.9869    0.9866   1310867

    accuracy                         0.9805   1810867
   macro avg     0.9759    0.9753    0.9756   1810867
weighted avg     0.9805    0.9805    0.9805   1810867

____________________
61423/61423 [==============================] - 737s 12ms/step
Matriz de Confusão Teste:  4
[[ 481841   18159]
 [  21336 1444171]]
Relatório de Classificação
              precision    recall  f1-score   support

           0     0.9576    0.9637    0.9606    500000
           1     0.9876    0.9854    0.9865   1465507

    accuracy                         0.9799   1965507
   macro avg     0.9726    0.9746    0.9736   1965507
weighted avg     0.9800    0.9799    0.9799   1965507

___________

## Teste Quarto Treino Incremental

In [11]:
model_bin = tf.keras.models.load_model('best_model_fase2')

majestic = pd.read_csv('./treino/majestic_ascii.csv')
majestic['69'] = 0
majestic['70'] = 0
majestic['71'] = 0

dga = pd.read_csv('./teste/df_teste3.csv')
dga = dga.drop('family', axis = 1)

dga.columns = majestic.columns
dga = dga.loc[~dga['domain'].isin(majestic['domain'])].reset_index(drop=True)

df_treino = pd.concat([majestic.sample(500000, random_state=1),dga], axis = 0, ignore_index=True)

X_train = df_treino.drop(['domain','label'], axis = 1)
y_train = df_treino['label']

X_train = X_train.to_numpy()
y_train = y_train.to_numpy()

best = tf.keras.callbacks.ModelCheckpoint(
    filepath='best_model_fase3',
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

model_bin.fit(X_train, y_train,
              epochs=10,
              callbacks = [best],
              validation_split = 0.2,
              batch_size=200)

Epoch 1/10
7244/7244 [==============================] - ETA: 0s - loss: 0.0249 - accuracy: 0.9922INFO:tensorflow:Assets written to: best_model_fase3/assets


INFO:tensorflow:Assets written to: best_model_fase3/assets


7244/7244 [==============================] - 298s 41ms/step - loss: 0.0249 - accuracy: 0.9922 - val_loss: 0.0347 - val_accuracy: 0.9865
Epoch 2/10
7244/7244 [==============================] - 293s 40ms/step - loss: 0.0171 - accuracy: 0.9950 - val_loss: 0.0581 - val_accuracy: 0.9796
Epoch 3/10
7244/7244 [==============================] - 295s 41ms/step - loss: 0.0137 - accuracy: 0.9961 - val_loss: 0.0465 - val_accuracy: 0.9851
Epoch 4/10
7244/7244 [==============================] - 295s 41ms/step - loss: 0.0113 - accuracy: 0.9969 - val_loss: 0.0528 - val_accuracy: 0.9836
Epoch 5/10
7244/7244 [==============================] - 295s 41ms/step - loss: 0.0098 - accuracy: 0.9973 - val_loss: 0.0692 - val_accuracy: 0.9797
Epoch 6/10
7244/7244 [==============================] - 295s 41ms/step - loss: 0.0087 - accuracy: 0.9977 - val_loss: 0.0815 - val_accuracy: 0.9786
Epoch 7/10
7244/7244 [==============================] - 294s 41ms/step - loss: 0.0079 - accuracy: 0.9979 - val_loss: 0.0737 - val

In [12]:
model_bin = tf.keras.models.load_model('best_model_fase3')
legit = majestic.loc[~majestic['domain'].isin(df_treino['domain'])].reset_index(drop=True)

for i in range (4,12):
    dga = pd.read_csv('./teste/df_teste'+str(i)+'.csv')
    dga = dga.drop('family', axis = 1)
    dga = dga.loc[~dga['domain'].isin(majestic['domain'])].reset_index(drop=True)

    dga.columns = legit.columns
    df_teste = pd.concat([legit,dga], axis = 0, ignore_index=True)

    X_test = df_teste.drop(['domain','label'], axis = 1)
    y_test = df_teste['label']

    X_test = X_test.to_numpy()
    y_test = y_test.to_numpy()

    with tf.device('/cpu:0'):
        predictions = model_bin.predict(X_test)
        predictions = np.where(predictions > 0.5, 1, 0)
    
    print('Matriz de Confusão Teste: ', str(i))
    print(confusion_matrix(y_test,predictions))

    print('Relatório de Classificação')
    print(classification_report(y_test,predictions, digits=4))
    print('____________________')

61423/61423 [==============================] - 785s 13ms/step
Matriz de Confusão Teste:  4
[[ 482327   17673]
 [  15118 1450389]]
Relatório de Classificação
              precision    recall  f1-score   support

           0     0.9696    0.9647    0.9671    500000
           1     0.9880    0.9897    0.9888   1465507

    accuracy                         0.9833   1965507
   macro avg     0.9788    0.9772    0.9780   1965507
weighted avg     0.9833    0.9833    0.9833   1965507

____________________
56719/56719 [==============================] - 697s 12ms/step
Matriz de Confusão Teste:  5
[[ 482327   17673]
 [  15055 1299933]]
Relatório de Classificação
              precision    recall  f1-score   support

           0     0.9697    0.9647    0.9672    500000
           1     0.9866    0.9886    0.9876   1314988

    accuracy                         0.9820   1814988
   macro avg     0.9782    0.9766    0.9774   1814988
weighted avg     0.9819    0.9820    0.9820   1814988

___________

## Teste Quinto Treino Incremental

In [13]:
model_bin = tf.keras.models.load_model('best_model_fase3')

majestic = pd.read_csv('./treino/majestic_ascii.csv')
majestic['69'] = 0
majestic['70'] = 0
majestic['71'] = 0

dga = pd.read_csv('./teste/df_teste4.csv')
dga = dga.drop('family', axis = 1)

dga.columns = majestic.columns
dga = dga.loc[~dga['domain'].isin(majestic['domain'])].reset_index(drop=True)

df_treino = pd.concat([majestic.sample(500000, random_state=1),dga], axis = 0, ignore_index=True)

X_train = df_treino.drop(['domain','label'], axis = 1)
y_train = df_treino['label']

X_train = X_train.to_numpy()
y_train = y_train.to_numpy()

best = tf.keras.callbacks.ModelCheckpoint(
    filepath='best_model_fase4',
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

model_bin.fit(X_train, y_train,
              epochs=10,
              callbacks = [best],
              validation_split = 0.2,
              batch_size=200)

Epoch 1/10
7861/7863 [============================>.] - ETA: 0s - loss: 0.0217 - accuracy: 0.9933

2024-01-21 18:18:23.565963: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 4.04GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


7863/7863 [==============================] - ETA: 0s - loss: 0.0217 - accuracy: 0.9933INFO:tensorflow:Assets written to: best_model_fase4/assets


INFO:tensorflow:Assets written to: best_model_fase4/assets


7863/7863 [==============================] - 323s 41ms/step - loss: 0.0217 - accuracy: 0.9933 - val_loss: 0.0337 - val_accuracy: 0.9879
Epoch 2/10
7863/7863 [==============================] - 322s 41ms/step - loss: 0.0155 - accuracy: 0.9955 - val_loss: 0.0396 - val_accuracy: 0.9860
Epoch 3/10
7863/7863 [==============================] - 325s 41ms/step - loss: 0.0123 - accuracy: 0.9966 - val_loss: 0.0496 - val_accuracy: 0.9844
Epoch 4/10
7863/7863 [==============================] - 325s 41ms/step - loss: 0.0104 - accuracy: 0.9972 - val_loss: 0.0581 - val_accuracy: 0.9816
Epoch 5/10
7863/7863 [==============================] - 323s 41ms/step - loss: 0.0090 - accuracy: 0.9976 - val_loss: 0.0703 - val_accuracy: 0.9810
Epoch 6/10
7863/7863 [==============================] - 322s 41ms/step - loss: 0.0081 - accuracy: 0.9979 - val_loss: 0.0766 - val_accuracy: 0.9807
Epoch 7/10
7863/7863 [==============================] - 323s 41ms/step - loss: 0.0073 - accuracy: 0.9981 - val_loss: 0.0845 - val

In [14]:
model_bin = tf.keras.models.load_model('best_model_fase4')
legit = majestic.loc[~majestic['domain'].isin(df_treino['domain'])].reset_index(drop=True)

for i in range (5,12):
    dga = pd.read_csv('./teste/df_teste'+str(i)+'.csv')
    dga = dga.drop('family', axis = 1)
    dga = dga.loc[~dga['domain'].isin(majestic['domain'])].reset_index(drop=True)

    dga.columns = legit.columns
    df_teste = pd.concat([legit,dga], axis = 0, ignore_index=True)

    X_test = df_teste.drop(['domain','label'], axis = 1)
    y_test = df_teste['label']

    X_test = X_test.to_numpy()
    y_test = y_test.to_numpy()

    with tf.device('/cpu:0'):
        predictions = model_bin.predict(X_test)
        predictions = np.where(predictions > 0.5, 1, 0)
    
    print('Matriz de Confusão Teste: ', str(i))
    print(confusion_matrix(y_test,predictions))

    print('Relatório de Classificação')
    print(classification_report(y_test,predictions, digits=4))
    print('____________________')

56719/56719 [==============================] - 756s 13ms/step
Matriz de Confusão Teste:  5
[[ 480738   19262]
 [  12262 1302726]]
Relatório de Classificação
              precision    recall  f1-score   support

           0     0.9751    0.9615    0.9683    500000
           1     0.9854    0.9907    0.9880   1314988

    accuracy                         0.9826   1814988
   macro avg     0.9803    0.9761    0.9781   1814988
weighted avg     0.9826    0.9826    0.9826   1814988

____________________
60739/60739 [==============================] - 778s 13ms/step
Matriz de Confusão Teste:  6
[[ 480738   19262]
 [  13779 1429854]]
Relatório de Classificação
              precision    recall  f1-score   support

           0     0.9721    0.9615    0.9668    500000
           1     0.9867    0.9905    0.9886   1443633

    accuracy                         0.9830   1943633
   macro avg     0.9794    0.9760    0.9777   1943633
weighted avg     0.9830    0.9830    0.9830   1943633

___________

## Teste Sexto Treino Incremental

In [15]:
model_bin = tf.keras.models.load_model('best_model_fase4')

majestic = pd.read_csv('./treino/majestic_ascii.csv')
majestic['69'] = 0
majestic['70'] = 0
majestic['71'] = 0

dga = pd.read_csv('./teste/df_teste5.csv')
dga = dga.drop('family', axis = 1)

dga.columns = majestic.columns
dga = dga.loc[~dga['domain'].isin(majestic['domain'])].reset_index(drop=True)

df_treino = pd.concat([majestic.sample(500000, random_state=1),dga], axis = 0, ignore_index=True)

X_train = df_treino.drop(['domain','label'], axis = 1)
y_train = df_treino['label']

X_train = X_train.to_numpy()
y_train = y_train.to_numpy()

best = tf.keras.callbacks.ModelCheckpoint(
    filepath='best_model_fase5',
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

model_bin.fit(X_train, y_train,
              epochs=10,
              callbacks = [best],
              validation_split = 0.2,
              batch_size=200)

Epoch 1/10
7260/7260 [==============================] - ETA: 0s - loss: 0.0220 - accuracy: 0.9933INFO:tensorflow:Assets written to: best_model_fase5/assets


INFO:tensorflow:Assets written to: best_model_fase5/assets


7260/7260 [==============================] - 300s 41ms/step - loss: 0.0220 - accuracy: 0.9933 - val_loss: 0.0375 - val_accuracy: 0.9865
Epoch 2/10
7260/7260 [==============================] - 298s 41ms/step - loss: 0.0158 - accuracy: 0.9955 - val_loss: 0.0489 - val_accuracy: 0.9828
Epoch 3/10
7260/7260 [==============================] - 297s 41ms/step - loss: 0.0128 - accuracy: 0.9965 - val_loss: 0.0484 - val_accuracy: 0.9851
Epoch 4/10
7260/7260 [==============================] - 295s 41ms/step - loss: 0.0107 - accuracy: 0.9971 - val_loss: 0.0574 - val_accuracy: 0.9822
Epoch 5/10
7260/7260 [==============================] - 295s 41ms/step - loss: 0.0093 - accuracy: 0.9975 - val_loss: 0.0608 - val_accuracy: 0.9842
Epoch 6/10
7260/7260 [==============================] - 295s 41ms/step - loss: 0.0081 - accuracy: 0.9978 - val_loss: 0.0764 - val_accuracy: 0.9815
Epoch 7/10
7260/7260 [==============================] - 295s 41ms/step - loss: 0.0076 - accuracy: 0.9980 - val_loss: 0.0637 - val

In [16]:
model_bin = tf.keras.models.load_model('best_model_fase5')
legit = majestic.loc[~majestic['domain'].isin(df_treino['domain'])].reset_index(drop=True)

for i in range (6,12):
    dga = pd.read_csv('./teste/df_teste'+str(i)+'.csv')
    dga = dga.drop('family', axis = 1)
    dga = dga.loc[~dga['domain'].isin(majestic['domain'])].reset_index(drop=True)

    dga.columns = legit.columns
    df_teste = pd.concat([legit,dga], axis = 0, ignore_index=True)

    X_test = df_teste.drop(['domain','label'], axis = 1)
    y_test = df_teste['label']

    X_test = X_test.to_numpy()
    y_test = y_test.to_numpy()

    with tf.device('/cpu:0'):
        predictions = model_bin.predict(X_test)
        predictions = np.where(predictions > 0.5, 1, 0)
    
    print('Matriz de Confusão Teste: ', str(i))
    print(confusion_matrix(y_test,predictions))

    print('Relatório de Classificação')
    print(classification_report(y_test,predictions, digits=4))
    print('____________________')

60739/60739 [==============================] - 783s 13ms/step
Matriz de Confusão Teste:  6
[[ 482527   17473]
 [  12667 1430966]]
Relatório de Classificação
              precision    recall  f1-score   support

           0     0.9744    0.9651    0.9697    500000
           1     0.9879    0.9912    0.9896   1443633

    accuracy                         0.9845   1943633
   macro avg     0.9812    0.9781    0.9796   1943633
weighted avg     0.9845    0.9845    0.9845   1943633

____________________
56582/56582 [==============================] - 699s 12ms/step
Matriz de Confusão Teste:  7
[[ 482527   17473]
 [  13762 1296860]]
Relatório de Classificação
              precision    recall  f1-score   support

           0     0.9723    0.9651    0.9686    500000
           1     0.9867    0.9895    0.9881   1310622

    accuracy                         0.9827   1810622
   macro avg     0.9795    0.9773    0.9784   1810622
weighted avg     0.9827    0.9827    0.9827   1810622

___________

## Sétimo Treino Incremental

In [7]:
model_bin = tf.keras.models.load_model('best_model_fase5')

majestic = pd.read_csv('./treino/majestic_ascii.csv')
majestic['69'] = 0
majestic['70'] = 0
majestic['71'] = 0

dga = pd.read_csv('./teste/df_teste6.csv')
dga = dga.drop('family', axis = 1)

dga.columns = majestic.columns
dga = dga.loc[~dga['domain'].isin(majestic['domain'])].reset_index(drop=True)

df_treino = pd.concat([majestic.sample(500000, random_state=1),dga], axis = 0, ignore_index=True)

X_train = df_treino.drop(['domain','label'], axis = 1)
y_train = df_treino['label']

X_train = X_train.to_numpy()
y_train = y_train.to_numpy()

best = tf.keras.callbacks.ModelCheckpoint(
    filepath='best_model_fase6',
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

model_bin.fit(X_train, y_train,
              epochs=10,
              callbacks = [best],
              validation_split = 0.2,
              batch_size=200)

2024-01-22 08:17:31.031464: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-22 08:17:31.232810: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-22 08:17:31.233167: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Epoch 1/10


2024-01-22 08:17:41.566569: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8902
2024-01-22 08:17:42.370168: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-01-22 08:17:42.393957: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f0e7821dfe0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-01-22 08:17:42.393978: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Laptop GPU, Compute Capability 8.6
2024-01-22 08:17:42.424221: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-01-22 08:17:42.603434: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifeti

7775/7775 [==============================] - ETA: 0s - loss: 0.0199 - accuracy: 0.9941

2024-01-22 08:22:39.564825: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.73GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-01-22 08:22:39.570415: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.73GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


INFO:tensorflow:Assets written to: best_model_fase6/assets


INFO:tensorflow:Assets written to: best_model_fase6/assets


7775/7775 [==============================] - 323s 41ms/step - loss: 0.0199 - accuracy: 0.9941 - val_loss: 0.0504 - val_accuracy: 0.9805
Epoch 2/10
7775/7775 [==============================] - ETA: 0s - loss: 0.0145 - accuracy: 0.9960INFO:tensorflow:Assets written to: best_model_fase6/assets


INFO:tensorflow:Assets written to: best_model_fase6/assets


7775/7775 [==============================] - 314s 40ms/step - loss: 0.0145 - accuracy: 0.9960 - val_loss: 0.0484 - val_accuracy: 0.9838
Epoch 3/10
7775/7775 [==============================] - 314s 40ms/step - loss: 0.0118 - accuracy: 0.9968 - val_loss: 0.0511 - val_accuracy: 0.9835
Epoch 4/10
7775/7775 [==============================] - 315s 41ms/step - loss: 0.0098 - accuracy: 0.9974 - val_loss: 0.0533 - val_accuracy: 0.9837
Epoch 5/10
7775/7775 [==============================] - 315s 40ms/step - loss: 0.0088 - accuracy: 0.9977 - val_loss: 0.0716 - val_accuracy: 0.9805
Epoch 6/10
7775/7775 [==============================] - 314s 40ms/step - loss: 0.0078 - accuracy: 0.9979 - val_loss: 0.0673 - val_accuracy: 0.9816
Epoch 7/10
7775/7775 [==============================] - 314s 40ms/step - loss: 0.0071 - accuracy: 0.9981 - val_loss: 0.0792 - val_accuracy: 0.9804
Epoch 8/10
7775/7775 [==============================] - 313s 40ms/step - loss: 0.0067 - accuracy: 0.9983 - val_loss: 0.0728 - val

In [8]:
model_bin = tf.keras.models.load_model('best_model_fase6')
legit = majestic.loc[~majestic['domain'].isin(df_treino['domain'])].reset_index(drop=True)

for i in range (7,12):
    dga = pd.read_csv('./teste/df_teste'+str(i)+'.csv')
    dga = dga.drop('family', axis = 1)
    dga = dga.loc[~dga['domain'].isin(majestic['domain'])].reset_index(drop=True)

    dga.columns = legit.columns
    df_teste = pd.concat([legit,dga], axis = 0, ignore_index=True)

    X_test = df_teste.drop(['domain','label'], axis = 1)
    y_test = df_teste['label']

    X_test = X_test.to_numpy()
    y_test = y_test.to_numpy()

    with tf.device('/cpu:0'):
        predictions = model_bin.predict(X_test)
        predictions = np.where(predictions > 0.5, 1, 0)
    
    print('Matriz de Confusão Teste: ', str(i))
    print(confusion_matrix(y_test,predictions))

    print('Relatório de Classificação')
    print(classification_report(y_test,predictions, digits=4))
    print('____________________')

56582/56582 [==============================] - 707s 12ms/step
Matriz de Confusão Teste:  7
[[ 481850   18150]
 [  12680 1297942]]
Relatório de Classificação
              precision    recall  f1-score   support

           0     0.9744    0.9637    0.9690    500000
           1     0.9862    0.9903    0.9883   1310622

    accuracy                         0.9830   1810622
   macro avg     0.9803    0.9770    0.9786   1810622
weighted avg     0.9829    0.9830    0.9829   1810622

____________________
61418/61418 [==============================] - 747s 12ms/step
Matriz de Confusão Teste:  8
[[ 481850   18150]
 [  13641 1451712]]
Relatório de Classificação
              precision    recall  f1-score   support

           0     0.9725    0.9637    0.9681    500000
           1     0.9877    0.9907    0.9892   1465353

    accuracy                         0.9838   1965353
   macro avg     0.9801    0.9772    0.9786   1965353
weighted avg     0.9838    0.9838    0.9838   1965353

___________

## Oitavo Treino Incremental

In [9]:
model_bin = tf.keras.models.load_model('best_model_fase6')

majestic = pd.read_csv('./treino/majestic_ascii.csv')
majestic['69'] = 0
majestic['70'] = 0
majestic['71'] = 0

dga = pd.read_csv('./teste/df_teste7.csv')
dga = dga.drop('family', axis = 1)

dga.columns = majestic.columns
dga = dga.loc[~dga['domain'].isin(majestic['domain'])].reset_index(drop=True)

df_treino = pd.concat([majestic.sample(500000, random_state=1),dga], axis = 0, ignore_index=True)

X_train = df_treino.drop(['domain','label'], axis = 1)
y_train = df_treino['label']

X_train = X_train.to_numpy()
y_train = y_train.to_numpy()

best = tf.keras.callbacks.ModelCheckpoint(
    filepath='best_model_fase7',
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

model_bin.fit(X_train, y_train,
              epochs=10,
              callbacks = [best],
              validation_split = 0.2,
              batch_size=200)

Epoch 1/10
7243/7243 [==============================] - ETA: 0s - loss: 0.0193 - accuracy: 0.9943INFO:tensorflow:Assets written to: best_model_fase7/assets


INFO:tensorflow:Assets written to: best_model_fase7/assets


7243/7243 [==============================] - 298s 41ms/step - loss: 0.0193 - accuracy: 0.9943 - val_loss: 0.0398 - val_accuracy: 0.9856
Epoch 2/10
7243/7243 [==============================] - ETA: 0s - loss: 0.0137 - accuracy: 0.9963INFO:tensorflow:Assets written to: best_model_fase7/assets


INFO:tensorflow:Assets written to: best_model_fase7/assets


7243/7243 [==============================] - 294s 41ms/step - loss: 0.0137 - accuracy: 0.9963 - val_loss: 0.0340 - val_accuracy: 0.9887
Epoch 3/10
7243/7243 [==============================] - 293s 40ms/step - loss: 0.0111 - accuracy: 0.9970 - val_loss: 0.0567 - val_accuracy: 0.9826
Epoch 4/10
7243/7243 [==============================] - 295s 41ms/step - loss: 0.0092 - accuracy: 0.9976 - val_loss: 0.0552 - val_accuracy: 0.9840
Epoch 5/10
7243/7243 [==============================] - 294s 41ms/step - loss: 0.0081 - accuracy: 0.9979 - val_loss: 0.0556 - val_accuracy: 0.9845
Epoch 6/10
7243/7243 [==============================] - 293s 41ms/step - loss: 0.0073 - accuracy: 0.9981 - val_loss: 0.0796 - val_accuracy: 0.9795
Epoch 7/10
7243/7243 [==============================] - 293s 40ms/step - loss: 0.0067 - accuracy: 0.9983 - val_loss: 0.0581 - val_accuracy: 0.9858
Epoch 8/10
7243/7243 [==============================] - 293s 40ms/step - loss: 0.0063 - accuracy: 0.9984 - val_loss: 0.0920 - val

In [10]:
model_bin = tf.keras.models.load_model('best_model_fase7')
legit = majestic.loc[~majestic['domain'].isin(df_treino['domain'])].reset_index(drop=True)

for i in range (8,12):
    dga = pd.read_csv('./teste/df_teste'+str(i)+'.csv')
    dga = dga.drop('family', axis = 1)
    dga = dga.loc[~dga['domain'].isin(majestic['domain'])].reset_index(drop=True)

    dga.columns = legit.columns
    df_teste = pd.concat([legit,dga], axis = 0, ignore_index=True)

    X_test = df_teste.drop(['domain','label'], axis = 1)
    y_test = df_teste['label']

    X_test = X_test.to_numpy()
    y_test = y_test.to_numpy()

    with tf.device('/cpu:0'):
        predictions = model_bin.predict(X_test)
        predictions = np.where(predictions > 0.5, 1, 0)
    
    print('Matriz de Confusão Teste: ', str(i))
    print(confusion_matrix(y_test,predictions))

    print('Relatório de Classificação')
    print(classification_report(y_test,predictions, digits=4))
    print('____________________')

61418/61418 [==============================] - 792s 13ms/step
Matriz de Confusão Teste:  8
[[ 476785   23215]
 [  10115 1455238]]
Relatório de Classificação
              precision    recall  f1-score   support

           0     0.9792    0.9536    0.9662    500000
           1     0.9843    0.9931    0.9887   1465353

    accuracy                         0.9830   1965353
   macro avg     0.9818    0.9733    0.9775   1965353
weighted avg     0.9830    0.9830    0.9830   1965353

____________________
56730/56730 [==============================] - 717s 13ms/step
Matriz de Confusão Teste:  9
[[ 476785   23215]
 [  10869 1304476]]
Relatório de Classificação
              precision    recall  f1-score   support

           0     0.9777    0.9536    0.9655    500000
           1     0.9825    0.9917    0.9871   1315345

    accuracy                         0.9812   1815345
   macro avg     0.9801    0.9727    0.9763   1815345
weighted avg     0.9812    0.9812    0.9812   1815345

___________

## Nono Treino Incremental

In [11]:
model_bin = tf.keras.models.load_model('best_model_fase7')

majestic = pd.read_csv('./treino/majestic_ascii.csv')
majestic['69'] = 0
majestic['70'] = 0
majestic['71'] = 0

dga = pd.read_csv('./teste/df_teste8.csv')
dga = dga.drop('family', axis = 1)

dga.columns = majestic.columns
dga = dga.loc[~dga['domain'].isin(majestic['domain'])].reset_index(drop=True)

df_treino = pd.concat([majestic.sample(500000, random_state=1),dga], axis = 0, ignore_index=True)

X_train = df_treino.drop(['domain','label'], axis = 1)
y_train = df_treino['label']

X_train = X_train.to_numpy()
y_train = y_train.to_numpy()

best = tf.keras.callbacks.ModelCheckpoint(
    filepath='best_model_fase8',
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

model_bin.fit(X_train, y_train,
              epochs=10,
              callbacks = [best],
              validation_split = 0.2,
              batch_size=200)

Epoch 1/10
7862/7862 [==============================] - ETA: 0s - loss: 0.0183 - accuracy: 0.9947INFO:tensorflow:Assets written to: best_model_fase8/assets


INFO:tensorflow:Assets written to: best_model_fase8/assets


7862/7862 [==============================] - 322s 41ms/step - loss: 0.0183 - accuracy: 0.9947 - val_loss: 0.0344 - val_accuracy: 0.9877
Epoch 2/10
7862/7862 [==============================] - 315s 40ms/step - loss: 0.0129 - accuracy: 0.9965 - val_loss: 0.0457 - val_accuracy: 0.9846
Epoch 3/10
7862/7862 [==============================] - 315s 40ms/step - loss: 0.0103 - accuracy: 0.9972 - val_loss: 0.0412 - val_accuracy: 0.9870
Epoch 4/10
7862/7862 [==============================] - 315s 40ms/step - loss: 0.0088 - accuracy: 0.9977 - val_loss: 0.0733 - val_accuracy: 0.9790
Epoch 5/10
7862/7862 [==============================] - 315s 40ms/step - loss: 0.0077 - accuracy: 0.9980 - val_loss: 0.0547 - val_accuracy: 0.9846
Epoch 6/10
7862/7862 [==============================] - 315s 40ms/step - loss: 0.0068 - accuracy: 0.9983 - val_loss: 0.0503 - val_accuracy: 0.9861
Epoch 7/10
7862/7862 [==============================] - 315s 40ms/step - loss: 0.0064 - accuracy: 0.9984 - val_loss: 0.0661 - val

In [12]:
model_bin = tf.keras.models.load_model('best_model_fase8')
legit = majestic.loc[~majestic['domain'].isin(df_treino['domain'])].reset_index(drop=True)

for i in range (9,12):
    dga = pd.read_csv('./teste/df_teste'+str(i)+'.csv')
    dga = dga.drop('family', axis = 1)
    dga = dga.loc[~dga['domain'].isin(majestic['domain'])].reset_index(drop=True)

    dga.columns = legit.columns
    df_teste = pd.concat([legit,dga], axis = 0, ignore_index=True)

    X_test = df_teste.drop(['domain','label'], axis = 1)
    y_test = df_teste['label']

    X_test = X_test.to_numpy()
    y_test = y_test.to_numpy()

    with tf.device('/cpu:0'):
        predictions = model_bin.predict(X_test)
        predictions = np.where(predictions > 0.5, 1, 0)
    
    print('Matriz de Confusão Teste: ', str(i))
    print(confusion_matrix(y_test,predictions))

    print('Relatório de Classificação')
    print(classification_report(y_test,predictions, digits=4))
    print('____________________')

56730/56730 [==============================] - 681s 12ms/step
Matriz de Confusão Teste:  9
[[ 480232   19768]
 [  11099 1304246]]
Relatório de Classificação
              precision    recall  f1-score   support

           0     0.9774    0.9605    0.9689    500000
           1     0.9851    0.9916    0.9883   1315345

    accuracy                         0.9830   1815345
   macro avg     0.9812    0.9760    0.9786   1815345
weighted avg     0.9830    0.9830    0.9830   1815345

____________________
61541/61541 [==============================] - 719s 12ms/step
Matriz de Confusão Teste:  10
[[ 480232   19768]
 [  13723 1455562]]
Relatório de Classificação
              precision    recall  f1-score   support

           0     0.9722    0.9605    0.9663    500000
           1     0.9866    0.9907    0.9886   1469285

    accuracy                         0.9830   1969285
   macro avg     0.9794    0.9756    0.9775   1969285
weighted avg     0.9829    0.9830    0.9830   1969285

__________

## Décimo treino Incremental

In [13]:
model_bin = tf.keras.models.load_model('best_model_fase8')

majestic = pd.read_csv('./treino/majestic_ascii.csv')
majestic['69'] = 0
majestic['70'] = 0
majestic['71'] = 0

dga = pd.read_csv('./teste/df_teste9.csv')
dga = dga.drop('family', axis = 1)

dga.columns = majestic.columns
dga = dga.loc[~dga['domain'].isin(majestic['domain'])].reset_index(drop=True)

df_treino = pd.concat([majestic.sample(500000, random_state=1),dga], axis = 0, ignore_index=True)

X_train = df_treino.drop(['domain','label'], axis = 1)
y_train = df_treino['label']

X_train = X_train.to_numpy()
y_train = y_train.to_numpy()

best = tf.keras.callbacks.ModelCheckpoint(
    filepath='best_model_fase9',
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

model_bin.fit(X_train, y_train,
              epochs=10,
              callbacks = [best],
              validation_split = 0.2,
              batch_size=200)

Epoch 1/10
7262/7262 [==============================] - ETA: 0s - loss: 0.0185 - accuracy: 0.9947INFO:tensorflow:Assets written to: best_model_fase9/assets


INFO:tensorflow:Assets written to: best_model_fase9/assets


7262/7262 [==============================] - 297s 41ms/step - loss: 0.0185 - accuracy: 0.9947 - val_loss: 0.0372 - val_accuracy: 0.9869
Epoch 2/10
7262/7262 [==============================] - 293s 40ms/step - loss: 0.0128 - accuracy: 0.9965 - val_loss: 0.0573 - val_accuracy: 0.9816
Epoch 3/10
7262/7262 [==============================] - 294s 40ms/step - loss: 0.0105 - accuracy: 0.9972 - val_loss: 0.0503 - val_accuracy: 0.9844
Epoch 4/10
7262/7262 [==============================] - 294s 40ms/step - loss: 0.0087 - accuracy: 0.9978 - val_loss: 0.0637 - val_accuracy: 0.9812
Epoch 5/10
7262/7262 [==============================] - 295s 41ms/step - loss: 0.0077 - accuracy: 0.9980 - val_loss: 0.0620 - val_accuracy: 0.9834
Epoch 6/10
7262/7262 [==============================] - 294s 40ms/step - loss: 0.0069 - accuracy: 0.9983 - val_loss: 0.0760 - val_accuracy: 0.9801
Epoch 7/10
7262/7262 [==============================] - 294s 41ms/step - loss: 0.0063 - accuracy: 0.9984 - val_loss: 0.0676 - val

In [14]:
model_bin = tf.keras.models.load_model('best_model_fase9')
legit = majestic.loc[~majestic['domain'].isin(df_treino['domain'])].reset_index(drop=True)

for i in range (10,12):
    dga = pd.read_csv('./teste/df_teste'+str(i)+'.csv')
    dga = dga.drop('family', axis = 1)
    dga = dga.loc[~dga['domain'].isin(majestic['domain'])].reset_index(drop=True)

    dga.columns = legit.columns
    df_teste = pd.concat([legit,dga], axis = 0, ignore_index=True)

    X_test = df_teste.drop(['domain','label'], axis = 1)
    y_test = df_teste['label']

    X_test = X_test.to_numpy()
    y_test = y_test.to_numpy()

    with tf.device('/cpu:0'):
        predictions = model_bin.predict(X_test)
        predictions = np.where(predictions > 0.5, 1, 0)
    
    print('Matriz de Confusão Teste: ', str(i))
    print(confusion_matrix(y_test,predictions))

    print('Relatório de Classificação')
    print(classification_report(y_test,predictions, digits=4))
    print('____________________')

61541/61541 [==============================] - 787s 13ms/step
Matriz de Confusão Teste:  10
[[ 480644   19356]
 [  11426 1457859]]
Relatório de Classificação
              precision    recall  f1-score   support

           0     0.9768    0.9613    0.9690    500000
           1     0.9869    0.9922    0.9896   1469285

    accuracy                         0.9844   1969285
   macro avg     0.9818    0.9768    0.9793   1969285
weighted avg     0.9843    0.9844    0.9843   1969285

____________________
57414/57414 [==============================] - 707s 12ms/step
Matriz de Confusão Teste:  11
[[ 480644   19356]
 [  12089 1325148]]
Relatório de Classificação
              precision    recall  f1-score   support

           0     0.9755    0.9613    0.9683    500000
           1     0.9856    0.9910    0.9883   1337237

    accuracy                         0.9829   1837237
   macro avg     0.9805    0.9761    0.9783   1837237
weighted avg     0.9828    0.9829    0.9828   1837237

_________

## Décimo primeiro treino Incremental

In [15]:
model_bin = tf.keras.models.load_model('best_model_fase9')

majestic = pd.read_csv('./treino/majestic_ascii.csv')
majestic['69'] = 0
majestic['70'] = 0
majestic['71'] = 0

dga = pd.read_csv('./teste/df_teste10.csv')
dga = dga.drop('family', axis = 1)

dga.columns = majestic.columns
dga = dga.loc[~dga['domain'].isin(majestic['domain'])].reset_index(drop=True)

df_treino = pd.concat([majestic.sample(500000, random_state=1),dga], axis = 0, ignore_index=True)

X_train = df_treino.drop(['domain','label'], axis = 1)
y_train = df_treino['label']

X_train = X_train.to_numpy()
y_train = y_train.to_numpy()

best = tf.keras.callbacks.ModelCheckpoint(
    filepath='best_model_fase10',
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

model_bin.fit(X_train, y_train,
              epochs=10,
              callbacks = [best],
              validation_split = 0.2,
              batch_size=200)

Epoch 1/10
7878/7878 [==============================] - ETA: 0s - loss: 0.0172 - accuracy: 0.9951

2024-01-22 14:55:39.204863: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 5.47GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


INFO:tensorflow:Assets written to: best_model_fase10/assets


INFO:tensorflow:Assets written to: best_model_fase10/assets


7878/7878 [==============================] - 321s 41ms/step - loss: 0.0172 - accuracy: 0.9951 - val_loss: 0.0305 - val_accuracy: 0.9886
Epoch 2/10
7877/7878 [============================>.] - ETA: 0s - loss: 0.0122 - accuracy: 0.9968INFO:tensorflow:Assets written to: best_model_fase10/assets


INFO:tensorflow:Assets written to: best_model_fase10/assets


7878/7878 [==============================] - 319s 40ms/step - loss: 0.0122 - accuracy: 0.9968 - val_loss: 0.0325 - val_accuracy: 0.9895
Epoch 3/10
7878/7878 [==============================] - 318s 40ms/step - loss: 0.0098 - accuracy: 0.9974 - val_loss: 0.0444 - val_accuracy: 0.9860
Epoch 4/10
7878/7878 [==============================] - 318s 40ms/step - loss: 0.0083 - accuracy: 0.9979 - val_loss: 0.0455 - val_accuracy: 0.9871
Epoch 5/10
7878/7878 [==============================] - 318s 40ms/step - loss: 0.0073 - accuracy: 0.9982 - val_loss: 0.0586 - val_accuracy: 0.9849
Epoch 6/10
7878/7878 [==============================] - 321s 41ms/step - loss: 0.0066 - accuracy: 0.9983 - val_loss: 0.0588 - val_accuracy: 0.9858
Epoch 7/10
7878/7878 [==============================] - 321s 41ms/step - loss: 0.0061 - accuracy: 0.9985 - val_loss: 0.0645 - val_accuracy: 0.9834
Epoch 8/10
7878/7878 [==============================] - 321s 41ms/step - loss: 0.0057 - accuracy: 0.9986 - val_loss: 0.0630 - val

In [16]:
model_bin = tf.keras.models.load_model('best_model_fase10')
legit = majestic.loc[~majestic['domain'].isin(df_treino['domain'])].reset_index(drop=True)

for i in range (11,12):
    dga = pd.read_csv('./teste/df_teste'+str(i)+'.csv')
    dga = dga.drop('family', axis = 1)
    dga = dga.loc[~dga['domain'].isin(majestic['domain'])].reset_index(drop=True)

    dga.columns = legit.columns
    df_teste = pd.concat([legit,dga], axis = 0, ignore_index=True)

    X_test = df_teste.drop(['domain','label'], axis = 1)
    y_test = df_teste['label']

    X_test = X_test.to_numpy()
    y_test = y_test.to_numpy()

    with tf.device('/cpu:0'):
        predictions = model_bin.predict(X_test)
        predictions = np.where(predictions > 0.5, 1, 0)
    
    print('Matriz de Confusão Teste: ', str(i))
    print(confusion_matrix(y_test,predictions))

    print('Relatório de Classificação')
    print(classification_report(y_test,predictions, digits=4))
    print('____________________')

57414/57414 [==============================] - 740s 13ms/step
Matriz de Confusão Teste:  11
[[ 479152   20848]
 [   9356 1327881]]
Relatório de Classificação
              precision    recall  f1-score   support

           0     0.9808    0.9583    0.9694    500000
           1     0.9845    0.9930    0.9888   1337237

    accuracy                         0.9836   1837237
   macro avg     0.9827    0.9757    0.9791   1837237
weighted avg     0.9835    0.9836    0.9835   1837237

____________________
